# Final Project: Flight Price Prediction

According to statistics from the Bureau of Transportation, over 853 million passengers traveled through U.S. airports in 2022. In 2020, there were 388 million passengers traveling and in 2021, there were 658 million passengers. The number of travelers has been steadily increasing year by year as the global aviation industry has expanded and demand for tourism has accelerated. However, not everyone in the world can afford to fly because of the high cost of air travel. We hope to give potential passengers and airlines an idea of the market demand and price of air travel. 


In our project we will answer the question:
     Are we able to find the cheapest flight price given certain criteria for flights?

## Part 1: Exploratory Data Analysis

First we need to import the necessary libraries for data analysis and preprocess our [kaggle dataset](https://www.kaggle.com/datasets/dilwong/flightprices). 

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#include imports for machine learning here

#to create the complex graph network
import networkx as nx

#pre-training, classification, fine tuning, testing, and generating
#when we do pretraining we need to make sure that the BERT model can make a distinction between rap styles. Based on the rhyme scheme. 

flight_data = pd.read_csv("data/itineraries.csv")
display(flight_data.summary())

#for data analysis of weekend prices v. weekday prices.
'''
TODO
    grab airline, day of journey, and flight price -> these parameters will change based on destination, so I will pick a set destination that is relatively large to 
'''
